# Chapter 07-04: 생성 모델 — VAE (Variational Autoencoder)

## 학습 목표
- Autoencoder와 VAE의 차이를 이해한다
- VAE의 ELBO 손실 함수와 재파라미터화 트릭을 이해한다
- MNIST 데이터로 VAE를 구현하고 학습한다
- 잠재 공간(Latent Space)을 시각화하고 보간(Interpolation)으로 새 이미지를 생성한다

## 목차
1. 수식 이해
2. 기본 Autoencoder 구현
3. VAE 확장 (재파라미터화 트릭)
4. 커스텀 VAE 손실 함수
5. 잠재 공간 시각화
6. 잠재 공간 보간으로 이미지 생성
7. 정리

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import matplotlib

# 한글 폰트 설정 (macOS)
matplotlib.rcParams['font.family'] = 'AppleGothic'
matplotlib.rcParams['axes.unicode_minus'] = False

print(f'TensorFlow 버전: {tf.__version__}')

# 재현성을 위한 시드 고정
tf.random.set_seed(42)
np.random.seed(42)

## 1. 수식 이해

### VAE ELBO (Evidence Lower BOund)

VAE는 데이터의 로그 우도(log-likelihood) $\log p(x)$를 직접 최적화하기 어려우므로,  
**하한(Lower Bound)**인 ELBO를 최대화하여 간접적으로 학습한다:

$$\mathcal{L} = \mathbb{E}_{q(z|x)}[\log p(x|z)] - D_{KL}(q(z|x) \| p(z))$$

| 항 | 이름 | 역할 |
|----|------|------|
| $\mathbb{E}_{q(z|x)}[\log p(x|z)]$ | 재구성 손실 | 디코더가 원본 $x$를 얼마나 잘 복원하는가 |
| $D_{KL}(q(z|x) \| p(z))$ | KL 발산 | 인코더의 잠재 분포를 표준 정규분포에 가깝게 |

### KL 발산 (가우시안 가정)

$q(z|x) = \mathcal{N}(\mu, \sigma^2)$, $p(z) = \mathcal{N}(0, I)$ 가정 시 닫힌 형태:

$$D_{KL} = -\frac{1}{2}\sum\left(1 + \log\sigma^2 - \mu^2 - \sigma^2\right)$$

### 재파라미터화 트릭 (Reparameterization Trick)

샘플링 연산은 역전파가 불가능하므로, 샘플링을 결정론적 연산으로 분리한다:

$$z = \mu + \sigma \odot \epsilon, \quad \epsilon \sim \mathcal{N}(0,I)$$

- $\epsilon$을 **표준 정규분포**에서 샘플링 (기울기 불필요)
- $\mu$와 $\sigma$는 인코더의 학습 가능한 출력 → 역전파 가능
- $\odot$: 원소별 곱셈(element-wise multiplication)

## 2. 기본 Autoencoder 구현 (MNIST)

In [ ]:
# MNIST 데이터 로드 및 전처리
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# 정규화 [0, 255] → [0, 1]
x_train = x_train.astype('float32') / 255.0
x_test  = x_test.astype('float32')  / 255.0

# 평탄화: (28, 28) → (784,)
x_train_flat = x_train.reshape(-1, 784)
x_test_flat  = x_test.reshape(-1, 784)

print(f'훈련 데이터: {x_train_flat.shape}')
print(f'테스트 데이터: {x_test_flat.shape}')

# ---- 기본 Autoencoder ----
LATENT_DIM_AE = 32  # 잠재 벡터 차원

# Encoder: 784 → 256 → 128 → latent_dim
ae_encoder_input = keras.Input(shape=(784,), name='ae_encoder_input')
ae_x = keras.layers.Dense(256, activation='relu')(ae_encoder_input)
ae_x = keras.layers.Dense(128, activation='relu')(ae_x)
ae_latent = keras.layers.Dense(LATENT_DIM_AE, activation='relu', name='latent')(ae_x)
ae_encoder = keras.Model(ae_encoder_input, ae_latent, name='AE_Encoder')

# Decoder: latent_dim → 128 → 256 → 784
ae_decoder_input = keras.Input(shape=(LATENT_DIM_AE,), name='ae_decoder_input')
ae_d = keras.layers.Dense(128, activation='relu')(ae_decoder_input)
ae_d = keras.layers.Dense(256, activation='relu')(ae_d)
ae_output = keras.layers.Dense(784, activation='sigmoid', name='ae_output')(ae_d)  # sigmoid → [0,1]
ae_decoder = keras.Model(ae_decoder_input, ae_output, name='AE_Decoder')

# Autoencoder 조합
ae_input = keras.Input(shape=(784,))
ae_encoded = ae_encoder(ae_input)
ae_reconstructed = ae_decoder(ae_encoded)
autoencoder = keras.Model(ae_input, ae_reconstructed, name='Autoencoder')

autoencoder.compile(
    optimizer='adam',
    loss='binary_crossentropy'  # 픽셀별 이진 교차 엔트로피
)

print('\n기본 Autoencoder 구조:')
ae_encoder.summary()

# 학습
print('\nAutoencoder 학습 시작...')
ae_history = autoencoder.fit(
    x_train_flat, x_train_flat,  # 입력과 타겟이 동일
    batch_size=256,
    epochs=10,
    validation_split=0.1,
    verbose=1
)

# 재구성 결과 시각화
n_samples = 8
test_samples = x_test_flat[:n_samples]
reconstructed = autoencoder.predict(test_samples, verbose=0)

fig, axes = plt.subplots(2, n_samples, figsize=(16, 4))
for i in range(n_samples):
    axes[0, i].imshow(test_samples[i].reshape(28, 28), cmap='gray')
    axes[0, i].axis('off')
    if i == 0:
        axes[0, i].set_title('원본', fontsize=10, pad=3)
    axes[1, i].imshow(reconstructed[i].reshape(28, 28), cmap='gray')
    axes[1, i].axis('off')
    if i == 0:
        axes[1, i].set_title('재구성', fontsize=10, pad=3)

plt.suptitle('Autoencoder 재구성 결과', fontsize=12, fontweight='bold')
plt.tight_layout()
plt.show()

## 3. VAE 확장 — Sampling 레이어(재파라미터화 트릭)

In [ ]:
LATENT_DIM = 2  # 2D 시각화를 위해 잠재 차원 = 2

# ---- Sampling 레이어 (재파라미터화 트릭 구현) ----
class SamplingLayer(keras.layers.Layer):
    """
    재파라미터화 트릭: z = mu + sigma * epsilon
    epsilon ~ N(0, I)
    """
    def call(self, inputs):
        mu, log_var = inputs  # 인코더로부터 평균과 로그 분산 수신
        
        # 배치 크기와 잠재 차원 추출
        batch = tf.shape(mu)[0]
        dim   = tf.shape(mu)[1]
        
        # 표준 정규 노이즈 샘플링 (역전파 불필요)
        epsilon = tf.random.normal(shape=(batch, dim))
        
        # z = mu + exp(0.5 * log_var) * epsilon
        # sigma = exp(0.5 * log_var) = sqrt(exp(log_var)) = sqrt(var)
        return mu + tf.exp(0.5 * log_var) * epsilon


# ---- VAE Encoder ----
encoder_input = keras.Input(shape=(784,), name='encoder_input')
enc_h1 = keras.layers.Dense(256, activation='relu')(encoder_input)
enc_h2 = keras.layers.Dense(128, activation='relu')(enc_h1)

# 평균(mu)과 로그 분산(log_var) 동시 출력
z_mu      = keras.layers.Dense(LATENT_DIM, name='z_mu')(enc_h2)       # 평균
z_log_var = keras.layers.Dense(LATENT_DIM, name='z_log_var')(enc_h2)  # 로그 분산

# 재파라미터화 샘플링
z = SamplingLayer(name='z_sampling')([z_mu, z_log_var])

# 인코더 모델 (mu, log_var, z 모두 출력)
vae_encoder = keras.Model(encoder_input, [z_mu, z_log_var, z], name='VAE_Encoder')

# ---- VAE Decoder ----
decoder_input = keras.Input(shape=(LATENT_DIM,), name='decoder_input')
dec_h1 = keras.layers.Dense(128, activation='relu')(decoder_input)
dec_h2 = keras.layers.Dense(256, activation='relu')(dec_h1)
decoder_output = keras.layers.Dense(784, activation='sigmoid', name='decoder_output')(dec_h2)

vae_decoder = keras.Model(decoder_input, decoder_output, name='VAE_Decoder')

print('VAE Encoder 구조:')
vae_encoder.summary()
print('\nVAE Decoder 구조:')
vae_decoder.summary()

## 4. 커스텀 VAE 손실 함수

In [ ]:
class VAE(keras.Model):
    """
    Variational Autoencoder
    커스텀 train_step으로 ELBO 손실(재구성 + KL) 계산
    """
    def __init__(self, encoder, decoder, **kwargs):
        super().__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        
        # 손실 추적기
        self.total_loss_tracker    = keras.metrics.Mean(name='total_loss')
        self.recon_loss_tracker    = keras.metrics.Mean(name='recon_loss')
        self.kl_loss_tracker       = keras.metrics.Mean(name='kl_loss')
    
    @property
    def metrics(self):
        return [self.total_loss_tracker, self.recon_loss_tracker, self.kl_loss_tracker]
    
    def train_step(self, data):
        x = data  # VAE는 입력 = 타겟
        
        with tf.GradientTape() as tape:
            # 인코더 실행: mu, log_var, z 추출
            z_mu, z_log_var, z = self.encoder(x, training=True)
            
            # 디코더 실행: z에서 이미지 재구성
            x_reconstructed = self.decoder(z, training=True)
            
            # 재구성 손실: 픽셀별 이진 교차 엔트로피의 합
            recon_loss = tf.reduce_mean(
                tf.reduce_sum(
                    keras.losses.binary_crossentropy(x, x_reconstructed),
                    axis=-1  # 784개 픽셀에 대해 합산
                )
            )
            
            # KL 발산 손실: -0.5 * sum(1 + log_var - mu^2 - exp(log_var))
            kl_loss = -0.5 * tf.reduce_mean(
                tf.reduce_sum(
                    1 + z_log_var - tf.square(z_mu) - tf.exp(z_log_var),
                    axis=-1  # 잠재 차원에 대해 합산
                )
            )
            
            # 전체 손실 = 재구성 손실 + KL 손실
            total_loss = recon_loss + kl_loss
        
        # 역전파
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        
        # 손실 업데이트
        self.total_loss_tracker.update_state(total_loss)
        self.recon_loss_tracker.update_state(recon_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        
        return {
            'total_loss': self.total_loss_tracker.result(),
            'recon_loss': self.recon_loss_tracker.result(),
            'kl_loss':    self.kl_loss_tracker.result()
        }


# VAE 인스턴스 생성 및 학습
vae = VAE(vae_encoder, vae_decoder, name='VAE')
vae.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-3))

print('VAE 학습 시작...')
vae_history = vae.fit(
    x_train_flat,
    batch_size=256,
    epochs=20,
    verbose=1
)

# 손실 곡선 시각화
fig, ax = plt.subplots(figsize=(10, 4))
ax.plot(vae_history.history['total_loss'], label='전체 손실', linewidth=2)
ax.plot(vae_history.history['recon_loss'], label='재구성 손실', linewidth=2, linestyle='--')
ax.plot(vae_history.history['kl_loss'],   label='KL 발산 손실', linewidth=2, linestyle=':')
ax.set_xlabel('에포크', fontsize=11)
ax.set_ylabel('손실', fontsize=11)
ax.set_title('VAE 학습 손실 (재구성 + KL 발산)', fontsize=12)
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 5. 잠재 공간 시각화 (2D 잠재 공간)

In [ ]:
# 테스트 데이터를 잠재 공간으로 인코딩
z_mu_test, z_log_var_test, z_test = vae_encoder.predict(x_test_flat, verbose=0)

print(f'테스트 잠재 벡터 shape: {z_test.shape}')  # (10000, 2)

fig, axes = plt.subplots(1, 2, figsize=(16, 7))

# --- 잠재 공간 산점도 (클래스별 색상) ---
ax = axes[0]
scatter = ax.scatter(
    z_test[:, 0], z_test[:, 1],
    c=y_test, cmap='tab10',
    alpha=0.5, s=5
)
plt.colorbar(scatter, ax=ax, label='숫자 클래스')
ax.set_xlabel('잠재 차원 1 ($z_1$)', fontsize=11)
ax.set_ylabel('잠재 차원 2 ($z_2$)', fontsize=11)
ax.set_title('VAE 2D 잠재 공간 (테스트 데이터)', fontsize=12)

# 각 클래스 중심점에 레이블 표시
for digit in range(10):
    mask = y_test == digit
    center_x = z_test[mask, 0].mean()
    center_y = z_test[mask, 1].mean()
    ax.annotate(str(digit), (center_x, center_y),
                fontsize=14, fontweight='bold', ha='center', va='center',
                color='white',
                bbox=dict(boxstyle='round,pad=0.3', facecolor='black', alpha=0.6))

# --- 잠재 공간 격자 → 디코딩된 이미지 ---
ax2 = axes[1]

# 잠재 공간의 범위 설정 ([-3, 3] × [-3, 3])
n_grid = 12
grid_range = np.linspace(-3, 3, n_grid)

# 격자 포인트 생성 → 디코딩 → 이미지 배치
canvas = np.zeros((28 * n_grid, 28 * n_grid))

for i, z2 in enumerate(grid_range[::-1]):
    for j, z1 in enumerate(grid_range):
        z_point = np.array([[z1, z2]])  # (1, 2)
        decoded = vae_decoder.predict(z_point, verbose=0)  # (1, 784)
        img = decoded[0].reshape(28, 28)
        canvas[i*28:(i+1)*28, j*28:(j+1)*28] = img

ax2.imshow(canvas, cmap='gray')
ax2.set_title(f'잠재 공간 격자 디코딩 ({n_grid}×{n_grid})', fontsize=12)
ax2.set_xlabel('$z_1$', fontsize=11)
ax2.set_ylabel('$z_2$', fontsize=11)
ax2.set_xticks([])
ax2.set_yticks([])

plt.suptitle('VAE 잠재 공간 분석', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

## 6. 잠재 공간 보간(Interpolation)으로 이미지 생성

In [ ]:
def interpolate_latent(z_start, z_end, n_steps=10):
    """
    두 잠재 벡터 사이를 선형 보간
    z = (1 - alpha) * z_start + alpha * z_end, alpha in [0, 1]
    """
    alphas = np.linspace(0, 1, n_steps)
    interpolated = np.array([
        (1 - alpha) * z_start + alpha * z_end
        for alpha in alphas
    ])  # (n_steps, latent_dim)
    return interpolated


# 각 숫자 쌍의 잠재 벡터 추출 (클래스 평균값 사용)
digit_pairs = [(0, 1), (2, 7), (3, 8), (4, 9)]
n_steps = 12

fig, axes = plt.subplots(len(digit_pairs), n_steps, figsize=(20, 7))

for row, (digit_a, digit_b) in enumerate(digit_pairs):
    # 각 클래스의 잠재 벡터 평균 계산
    z_a = z_mu_test[y_test == digit_a].mean(axis=0)  # (2,)
    z_b = z_mu_test[y_test == digit_b].mean(axis=0)  # (2,)
    
    # 선형 보간
    z_interp = interpolate_latent(z_a, z_b, n_steps)  # (n_steps, 2)
    
    # 디코딩
    decoded_images = vae_decoder.predict(z_interp, verbose=0)  # (n_steps, 784)
    
    for col in range(n_steps):
        ax = axes[row, col]
        ax.imshow(decoded_images[col].reshape(28, 28), cmap='gray')
        ax.axis('off')
        
        # 시작/끝 레이블
        if col == 0:
            ax.set_title(f'{digit_a}', fontsize=11, fontweight='bold', color='blue')
        elif col == n_steps - 1:
            ax.set_title(f'{digit_b}', fontsize=11, fontweight='bold', color='red')

plt.suptitle('잠재 공간 선형 보간: 두 숫자 사이의 연속적 변환', fontsize=13, fontweight='bold')
plt.tight_layout()
plt.show()

print('보간 결과: 잠재 공간이 연속적이고 의미있는 구조를 가짐을 확인')

# 순수 생성: 표준 정규 분포에서 샘플링
print('\n랜덤 생성 이미지 (z ~ N(0,I)):')
n_generated = 16
z_random = np.random.randn(n_generated, LATENT_DIM)  # 표준 정규 샘플링
generated = vae_decoder.predict(z_random, verbose=0)

fig, axes = plt.subplots(2, 8, figsize=(16, 5))
for i, ax in enumerate(axes.flat):
    ax.imshow(generated[i].reshape(28, 28), cmap='gray')
    ax.axis('off')

plt.suptitle('VAE 랜덤 생성 이미지 ($z \\sim \\mathcal{N}(0,I)$)', fontsize=12, fontweight='bold')
plt.tight_layout()
plt.show()

## 7. 정리

### Autoencoder vs VAE 비교

| 구분 | Autoencoder | VAE |
|------|------------|-----|
| 잠재 공간 | 비구조적 (점) | 확률 분포 $\mathcal{N}(\mu, \sigma^2)$ |
| 손실 함수 | 재구성 손실만 | 재구성 손실 + KL 발산 |
| 생성 가능성 | 낮음 (새 점 샘플링 어려움) | 높음 ($z \sim \mathcal{N}(0,I)$ 샘플링) |
| 잠재 공간 연속성 | 보장 없음 | 연속적, 매끄러운 보간 가능 |
| 역전파 가능성 | 가능 | 재파라미터화 트릭으로 가능 |

### 핵심 개념 요약

- **ELBO**: 직접 최적화 불가능한 로그 우도의 하한 → 최대화
- **재파라미터화 트릭**: 샘플링을 결정론적 연산으로 분리하여 역전파 가능
- **KL 발산**: 잠재 분포를 표준 정규분포로 정규화 → 잠재 공간의 연속성 보장
- **보간**: 잠재 공간이 매끄러우므로 두 점 사이의 선형 보간이 의미있는 전환 생성

### 다음 단계

**Chapter 07-05: GAN (Generative Adversarial Networks)** — 생성자와 판별자의 적대적 학습으로 더욱 사실적인 이미지를 생성한다.